In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as colors
import pandas as pd
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.decomposition import PCA
%matplotlib inline
import random
from sklearn import preprocessing
import scipy.sparse as sparse
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import graphlab as gl

## Overview

This notebook allows for one to see the % of the total Metadata Products come from a particular category, and the % of Reviews come from a particular Category.

In [2]:
filename = "datasets/reviews.csv"
df = pd.read_csv(filename)

In [3]:
df.head()

,Unnamed: 0,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


In [4]:
filename2 = "datasets/meta_Kindle_Store.csv"
df2 = pd.read_csv(filename2)

//anaconda/envs/gl-env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
df2.head(10)

,asin,description,price,imUrl,related,categories,title,salesRank,brand
0,1603420304,In less time and for less money than it takes ...,7.69,http://ecx.images-amazon.com/images/I/51IEqPrF...,"{'also_viewed': ['B001OLRKLQ', 'B004J35JIC', '...","[['Books', 'Cookbooks, Food & Wine', 'Quick & ...",NaN,NaN,NaN
1,B0002IQ15S,This universal DC adapter powers/charges porta...,19.99,http://ecx.images-amazon.com/images/I/21QFJM28...,"{'also_viewed': ['B00511PS3C', 'B000PI17MM', '...","[['Kindle Store', 'Kindle Accessories', 'Power...",Mobility IGO AUTOPOWER 3000 SERIES ( PS0221-10 ),{},NaN
2,B000F83SZQ,NaN,0.00,http://ecx.images-amazon.com/images/I/51yLqHe%...,"{'also_bought': ['B0080H1C0W', 'B00LK4ZKOG', '...","[['Books', 'Literature & Fiction'], ['Books', ...",NaN,NaN,NaN
3,B000F83TEQ,NaN,NaN,http://ecx.images-amazon.com/images/I/2136NBNV...,"{'also_bought': ['B00IS81LFO', 'B000FA5T6A', '...","[['Books', 'Literature & Fiction'], ['Books', ...",NaN,NaN,NaN
4,B000F83STC,NaN,NaN,http://g-ecx.images-amazon.com/images/G/01/x-s...,NaN,"[['Books', 'Literature & Fiction', 'Erotica'],...",NaN,NaN,NaN
5,B000FA5RE4,Updated 2003 version with corrections,1.99,http://ecx.images-amazon.com/images/I/21XTM6V0...,"{'also_viewed': ['B001GX3ZWA', 'B009GI3S1A', '...","[['Books', 'Reference', 'Dictionaries & Thesau...",NaN,NaN,NaN
6,B000FA5NSO,NaN,NaN,http://ecx.images-amazon.com/images/I/51N45B6j...,"{'also_viewed': ['B000FBF81K', 'B000FA5KKA']}","[['Books', 'Literature & Fiction'], ['Books', ...",NaN,NaN,NaN
7,B000FA5UXC,Madoc Tamlin is a man with an unusual problem....,NaN,http://ecx.images-amazon.com/images/I/51q4iur5...,"{'also_bought': ['B004SPL1I8', 'B00942QL3K', '...","[['Books', 'Literature & Fiction'], ['Books', ...",NaN,NaN,NaN
8,B000FA5T4W,NaN,12.46,http://ecx.images-amazon.com/images/I/51eWyBr8...,"{'also_bought': ['B004RKXHGS', 'B004ULMJH2', '...","[['Books', 'Business & Money', 'Accounting'], ...",NaN,NaN,NaN
9,B000FA5SHK,NaN,NaN,http://ecx.images-amazon.com/images/I/51c7mqOR...,NaN,"[['Books', 'Science & Math', 'Behavioral Scien...",NaN,NaN,NaN


In [200]:
topicStr= "Science Fiction"

In [201]:
df3 = pd.DataFrame({"asin":df2.asin,"topic":df2.categories.str.contains(topicStr)})

In [202]:
df3.head()

,asin,topic
0,1603420304,False
1,B0002IQ15S,False
2,B000F83SZQ,False
3,B000F83TEQ,False
4,B000F83STC,False


In [203]:
float(sum(df3.topic))/df3.shape[0]

0.12415861900796407

In [204]:
df4 = pd.DataFrame({"asin":df.asin})

In [205]:
df4.head()

,asin
0,B000F83SZQ
1,B000F83SZQ
2,B000F83SZQ
3,B000F83SZQ
4,B000F83SZQ


In [206]:
df4 =df4.merge(df3, on='asin', how='left')

In [207]:
float(sum(df4.topic))/df4.shape[0]

0.1512865108449969

In [52]:
print df.shape
print df2.shape
print df3.shape
print df4.shape


(982619, 10)
(434702, 9)
(434702, 2)
(982619, 2)
